In [12]:
#for a general introduction to scraping, please see https://www.digitalocean.com/community/tutorials/how-to-scrape-web-pages-with-beautiful-soup-and-python-3

pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [28]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
import re
import os

In [29]:
page = requests.get('https://www.royal.uk/speeches')

In [30]:
# Create a BeautifulSoup object
soup_main = BeautifulSoup(page.text, 'html.parser')

In [31]:
#get the list of members of the royal house
list_of_members=[]
list_complete=soup_main.find(class_="form-item form-type-select form-item-mrf").find_all('option')
for i in list_complete:
    list_of_members.append((i.text, i['value']))
list_of_members.pop(0)

('All Members of the Royal Family', '')

In [32]:
for i in list_of_members:
    name=re.sub(' ', '', i[0])
    if not os.path.exists('C:/Users/u0140177/OneDrive - KU Leuven/DH_commons_presentation/speeches/'+name):
        os.makedirs('C:/Users/u0140177/OneDrive - KU Leuven/DH_commons_presentation/speeches/'+name)
    page=requests.get('https://www.royal.uk/speeches?text=&mrf='+i[1]+'&date[min][date]=1%2F01%2F1990&date[max][date]=15%2F04%2F2021&id=')
    soup=BeautifulSoup(page.text, 'html.parser')
    if soup.find(class_="pager-last last") is not None:
        nb_last_page=soup.find(class_="pager-last last").find('a').get('href').split('page=')[1]
    else:
        nb_last_page=0
    #working with the first page
    list_of_links_with_date=[]
    link_infos= soup.find_all(class_='tbm-fields-container views-fieldset')
    for item in link_infos:
        link_ref=item.find('a')
        link = 'https://www.royal.uk' + link_ref.get('href')
        date=item.find(class_="date-display-single")
        year=date.text.split()[-1]
        list_of_links_with_date.append((link, year))
    #if there are toher results pages, I access them
    if nb_last_page!=0:
        list_of_pages=[]
        for j in range(1, int(nb_last_page)+1):
            list_of_pages.append('https://www.royal.uk/speeches?text=&mrf='+i[1]+'&date[min][date]=01/01/1990&date[max][date]=15/04/2021&id=&page='+str(j)) 
        for m in list_of_pages:
            page = requests.get(m)
            soup = BeautifulSoup(page.text, 'html.parser')
            link_infos= soup.find_all(class_='tbm-fields-container views-fieldset')
            for item in link_infos:
                link_ref=item.find('a')
                link = 'https://www.royal.uk' + link_ref.get('href')
                date=item.find(class_="date-display-single")
                year=date.text.split()[-1]
                list_of_links_with_date.append((link, year))
    #I print the list of links of the articles+year for every member of the Royal family
    file = open('C:/Users/u0140177/OneDrive - KU Leuven/DH_commons_presentation/speeches/'+name+'/'+name+'links_and_dates.csv', 'w')
    f=csv.writer(file)
    for l in range(len(list_of_links_with_date)):
        f.writerow([list_of_links_with_date[l][0], list_of_links_with_date[l][1]])
    file.close()          
        


In [33]:
#using the list of links created in the previous cell, I download the speeches
for i in list_of_members:
    name=re.sub(' ', '', i[0])
    file_links='C:/Users/u0140177/OneDrive - KU Leuven/DH_commons_presentation/speeches/'+name+'/'+name+'links_and_dates.csv'
    if os.stat(file_links).st_size != 0:
        df=pd.read_csv(file_links, names=["link", "year"])
        for index, row in df.iterrows():
            title='x'
            content='x'
            page = requests.get(row["link"])
            soup = BeautifulSoup(page.text, 'html.parser')
            last_links = soup.find(class_="tbm-share-links share")
            last_links.decompose()
            if soup.find(class_="container article").find('h1') is not None:
                title=soup.find(class_="container article").find('h1').text
            if soup.find(class_="block block-system").find(class_="content") is not None:
                content= soup.find(class_="block block-system").find(class_="content").find_all('p')
                content_text=''
                for j in content[:-1]:
                    content_text+=j.text
            f=open('C:/Users/u0140177/OneDrive - KU Leuven/DH_commons_presentation/speeches/'+name+'/'+name+'_'+str(row["year"])+'_'+title[0:20]+'.txt', 'w', encoding="utf-8")
            f.write(title+"\n"+content_text)
            f.close()
